#### (please uncomment these and install the packages if needed)

In [1]:
# !pip install pandas
# !pip install numpy
# !pip install scikit-learn
# !pip install onnxruntime

In [2]:
import pandas as pd
import numpy as np
import onnxruntime

### loading from 'test_cases.csv'

In [3]:
test_df = pd.read_csv('test_cases.csv')
test_df

,hash,REQUEST_IGNORE_BATTERY_OPTIMIZATIONS-ins,REQUEST_IGNORE_BATTERY_OPTIMIZATIONS-run,SYSTEM_ALERT_WINDOW-ins,SYSTEM_ALERT_WINDOW-run,WRITE_SETTINGS-ins,WRITE_SETTINGS-run,ACCESS_COARSE_LOCATION-ins,ACCESS_COARSE_LOCATION-run,ACCESS_FINE_LOCATION-ins,...,WAKE_LOCK-run,WRITE_CALENDAR-ins,WRITE_CALL_LOG-ins,WRITE_CALL_LOG-run,WRITE_CONTACTS-ins,WRITE_CONTACTS-run,WRITE_EXTERNAL_STORAGE-ins,WRITE_EXTERNAL_STORAGE-run,WRITE_SYNC_SETTINGS-ins,label
0,9E32ECE0C168894EAC864206A694815DF11E9E51B76C1D...,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,3AA728E5574D559F192BFCB7A4C5C70ED7A65F12BA0170...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,D846252A52FD755FB3D0248A1031226377F40F2B4568AC...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,D56CD846CF8E9FF38CB0679EEDEB0BBB761552327AC9D4...,0,0,0,0,0,0,1,0,1,...,0,0,0,0,0,0,1,1,0,0
4,411DDCDB980ACDFB3C59B45282EDB8465599160DED8C52...,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
700,1069447316123a320b1c043db61d62e2a6360399347c29...,0,1,0,1,0,0,1,1,1,...,1,0,0,1,0,1,1,1,0,1
701,ce4186a07a91cee08f3930247fbdb41df09972343f58c8...,0,1,0,1,0,0,1,1,1,...,1,0,0,1,0,1,1,1,0,1
702,3e448d8dbb9807acab4e3ecf0000f9a182ed77c18e93c3...,0,1,0,1,0,0,0,1,0,...,1,0,0,1,0,1,1,1,0,1
703,c712a469c548e12bd3dfe93f6779c864258fc86a55e64d...,0,1,0,1,0,0,0,1,0,...,1,0,0,1,0,1,1,1,0,1


#### label information about the test cases, 0 means benign and 1 means Vishing

In [4]:
y_test = test_df['label']
print(y_test)

0      0
1      0
2      0
3      0
4      0
      ..
700    1
701    1
702    1
703    1
704    1
Name: label, Length: 705, dtype: int64


#### the same cases as those used for evaluating VishielDroid are used here

In [5]:
print('# benign cases : ' + str( (test_df['label'] == 0).sum() ) )
print('# Vishing cases : ' + str( (test_df['label'] == 1).sum() ) ) 

# benign cases : 475
# Vishing cases : 230


### loading from 'trained_SVM.onnx'

In [6]:
model_path = "trained_SVM.onnx"
ort_session = onnxruntime.InferenceSession(model_path)

In [7]:
input_name = ort_session.get_inputs()[0].name
print(input_name)

int64_input


### testing the model with the benign/Vishing cases from 2023

In [8]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

test_df = test_df.drop('hash', axis=1)

test_cases = np.array( test_df.iloc[:, :-1] )
input_data = np.array( test_cases , dtype=np.int64)


### running the model 10 times and getting the average metrics

In [9]:

precision_history = []
recall_history = []

accuracy_history = []
f1_history = []


for i in range(10):

    output = ort_session.run(None, {input_name: input_data})
    y_pred = output[0]

    # Precision
    cur_precision = precision_score(y_test, y_pred)
    precision_history.append(cur_precision)

    # Recall
    cur_recall = recall_score(y_test, y_pred)
    recall_history.append(cur_recall)

    # Accuracy
    cur_acc = accuracy_score(y_test, y_pred)
    accuracy_history.append(cur_acc)

    # F1-score
    cur_f1 = f1_score(y_test, y_pred)
    f1_history.append(cur_f1)
        

print('Mean Precision : ' + str( round( np.mean(precision_history) * 100 , 2 ) ) )
print('Mean Recall : ' + str(  round( np.mean(recall_history) * 100 , 2 ) ) )
print('Mean Accuracy : ' + str( round( np.mean(accuracy_history) * 100, 2 ) ) )
print('Mean F1-scroe : ' + str(  round( np.mean(f1_history) * 100 , 2 ) ) ) 


Mean Precision : 100.0
Mean Recall : 99.57
Mean Accuracy : 99.86
Mean F1-scroe : 99.78
